# Feature Engineering - One Hot Encoding
Here we prep the dataframe for modelling, converting all the 

In [2]:
import pandas as pd
import numpy as np
import os
import json
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
import sys
os.chdir('..')
cwd = os.getcwd()
data_dir = cwd +'/data/' #path to the data folder which contains a 'pre-processed' folder within it
print(data_dir) #check that it's correct

/Users/admin/Documents/Jobs/Datatonic/datatonic-challenge/data/


In [3]:
#loading the data from the pickled file
data = pd.read_pickle(data_dir + 'pre-processed/movie_details_neat.pkl')

#loading id map and revrse map
with open(data_dir+'pre-processed/id_maps.json', 'r') as f:
    id_maps = json.load(f)
with open(data_dir+'pre-processed/id_maps_reverse.json', 'r') as f:
    id_maps_reverse = json.load(f)
data.head() #quick look at the data

,budget,id,popularity,revenue,runtime,vote_average,vote_count,genres,keywords,original_language,original_title,overview,production_companies,production_countries,release_date,spoken_languages,movie_id,cast,crew
0,105000000,64682,61.196071,351040419,143.0,7.3,3769,"{'id': [18, 10749], 'name': ['Drama', 'Romance']}","{'id': [818, 1326, 1523, 3929, 209714], 'name'...",en,The Great Gatsby,An adaptation of F. Scott Fitzgerald's Long Is...,"{'name': ['Village Roadshow Pictures', 'Bazmar...","{'iso_3166_1': ['US', 'AU'], 'name': ['United ...",2013-05-10,"{'iso_639_1': ['en'], 'name': ['English']}",64682,"{'cast_id': [2, 5, 3, 8, 6, 4, 7, 22, 13, 23, ...","{'credit_id': ['52fe46e3c3a368484e0a982d', '52..."
1,150000000,9543,62.169881,335154643,116.0,6.2,2317,"{'id': [12, 14, 28, 10749], 'name': ['Adventur...","{'id': [1241, 1965, 12653, 12654, 12655, 41645...",en,Prince of Persia: The Sands of Time,A rogue prince reluctantly joins forces with a...,"{'name': ['Walt Disney Pictures', 'Jerry Bruck...","{'iso_3166_1': ['US'], 'name': ['United States...",2010-05-19,"{'iso_639_1': ['en'], 'name': ['English']}",9543,"{'cast_id': [5, 2, 7, 4, 6, 8, 9, 10, 26, 27, ...","{'credit_id': ['567e74d4c3a36860e9008e46', '52..."
2,140000000,5174,22.571780,258022233,91.0,6.1,783,"{'id': [28, 35, 80, 53], 'name': ['Action', 'C...","{'id': [1704], 'name': ['ambassador']}",en,Rush Hour 3,After an attempted assassination on Ambassador...,"{'name': ['New Line Cinema'], 'id': [12]}","{'iso_3166_1': ['US'], 'name': ['United States...",2007-08-08,"{'iso_639_1': ['la', 'en', 'fr', 'ja', 'zh'], ...",5174,"{'cast_id': [2, 3, 4, 5, 6, 7, 8, 9, 26, 27, 2...","{'credit_id': ['52fe43fac3a36847f807b5bd', '52..."
3,145000000,1735,60.034162,401128639,112.0,5.2,1387,"{'id': [12, 28, 14], 'name': ['Adventure', 'Ac...",{},en,The Mummy: Tomb of the Dragon Emperor,"Archaeologist Rick O'Connell travels to China,...","{'name': ['Universal Pictures', 'China Film Co...","{'iso_3166_1': ['DE', 'US'], 'name': ['Germany...",2008-07-01,"{'iso_639_1': ['en', 'zh', 'sa'], 'name': ['En...",1735,"{'cast_id': [1, 2, 8, 12, 13, 14, 15, 16, 17, ...","{'credit_id': ['52fe4312c3a36847f80384c5', '52..."
4,27000000,79698,2.418535,0,109.0,4.8,34,"{'id': [28, 12, 878, 10749], 'name': ['Action'...",{},en,The Lovers,The Lovers is an epic romance time travel adve...,"{'name': ['Corsan', 'Bliss Media', 'Limelight ...","{'iso_3166_1': ['AU', 'BE', 'IN'], 'name': ['A...",2015-02-13,"{'iso_639_1': ['en'], 'name': ['English']}",79698,"{'cast_id': [11, 13, 22, 17, 14, 15, 16, 18, 1...","{'credit_id': ['52fe49e0c3a368484e145067', '57..."


### Generating new keywords
A number of movies were missing keywords. This function takes the movie overview, breaks it up into words, compares if any of those words are existing keywords in id_maps, and if they are, they are added to the keywords dictionary. This enhances already existing keywords and adds new ones where there were none previously

In [4]:
def generate_keywords(id_maps, id_maps_reverse, df, row):        
    book_id = id_maps_reverse['keywords']['book']
    novel_id = id_maps_reverse['keywords']['based on novel']
    graphic_novel_id = id_maps_reverse['keywords']['based on graphic novel']
    comic_id = id_maps_reverse['keywords']['marvel comic']
    #print(book_id, novel_id, graphic_novel_id, comic_id)

    id_maps['keywords'][novel_id] = 'novel'#rename based on novel to just novel to be more general
    id_maps['keywords'][comic_id] = 'comic' #rename marvel comic to just comic to be more general
    
    #add new entries in the reverse map to reflect changes
    id_maps_reverse['keywords']['novel'] = novel_id
    id_maps_reverse['keywords']['comic'] = comic_id
    

    word_list = df['overview'][row].split()
    word_id = []
    word = []
    for i in word_list:
        i = i.replace(',', '')
        i = i.replace('.', '')
        present = i in list(id_maps['keywords'].values())
        if present is True:
            word_id.append(id_maps_reverse['keywords'][i])
            word.append(i)

    new_keywords = {'id'   : word_id,
                    'name' : word}
    
    return new_keywords

data_copy = data.copy()
print("Number of rows: {}".format(len(data_copy)))
for i in range(len(data_copy)):
    if len(data_copy['keywords'][i]) == 0:
        data_copy['keywords'][i] = generate_keywords(id_maps, id_maps_reverse, data_copy, i)

Number of rows: 4326


/Users/admin/.local/share/virtualenvs/datatonic-challenge-ErycU493/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
"""

function below goes through the columns listed in columns_to_convert and creates a new column, with each entry
being a list of words from the original column (so essentially getting rid of ids)

this is useful for encoding later

"""


columns_to_convert = ['genres', 'keywords', 'production_companies', 'production_countries',
            'spoken_languages', 'cast', 'crew']

def get_ids(df, columns_to_convert):
    new_columns = []
    df['lead_male']   = 0 
    df['lead_female'] = 0
    df['lead2_male']  = 0
    df['lead2_female']= 0
    for column in columns_to_convert:
        print(column)
        new_column_name = column + '_name'
        new_columns.append(new_column_name)
        df[new_column_name] = np.nan
        df[new_column_name] = df[new_column_name].astype(object)
        for i in range(len(df)):
            try:
                if column is 'production_countries':
                    name = df[column][i]['name']
                elif column is 'spoken_languages':
                    name = df[column][i]['iso_639_1']
                    
                #adding columns for lead genders
                elif column is 'cast':
                    name = df[column][i]['name']
                    try:
                        lead_gender = df[column][i]['gender'][0]
                        second_lead_gender = df[column][i]['gender'][1]
                        if lead_gender == 2:
                            df['lead_male'][i] = 1
                        if lead_gender == 1:
                            df['lead_female'][i] = 1
                        if second_lead_gender == 2:
                            df['lead2_male'][i]  = 1
                        if second_lead_gender == 1:
                            df['lead2_female'][i] = 1
                    except IndexError:
                        pass
                        

                else:
                    name = df[column][i]['name']
                    
                df.at[i, new_column_name] = name
                
            except KeyError:
                length = len(df[column][i])
                if length == 0:
                    name = []
                    df.at[i, new_column_name] = name
    return df, new_columns
        
    
    
data_copy, new_columns = get_ids(data_copy, columns_to_convert)
pd.set_option('display.max_columns', None)
data_copy.head()

genres
keywords
production_companies
production_countries
spoken_languages
cast


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to b

crew


,budget,id,popularity,revenue,runtime,vote_average,vote_count,genres,keywords,original_language,original_title,overview,production_companies,production_countries,release_date,spoken_languages,movie_id,cast,crew,lead_male,lead_female,lead2_male,lead2_female,genres_name,keywords_name,production_companies_name,production_countries_name,spoken_languages_name,cast_name,crew_name
0,105000000,64682,61.196071,351040419,143.0,7.3,3769,"{'id': [18, 10749], 'name': ['Drama', 'Romance']}","{'id': [818, 1326, 1523, 3929, 209714], 'name'...",en,The Great Gatsby,An adaptation of F. Scott Fitzgerald's Long Is...,"{'name': ['Village Roadshow Pictures', 'Bazmar...","{'iso_3166_1': ['US', 'AU'], 'name': ['United ...",2013-05-10,"{'iso_639_1': ['en'], 'name': ['English']}",64682,"{'cast_id': [2, 5, 3, 8, 6, 4, 7, 22, 13, 23, ...","{'credit_id': ['52fe46e3c3a368484e0a982d', '52...",1,0,1,0,"[Drama, Romance]","[based on novel, infidelity, obsession, hope, 3d]","[Village Roadshow Pictures, Bazmark Films, War...","[United States of America, Australia]",[en],"[Leonardo DiCaprio, Tobey Maguire, Carey Mulli...","[Ronna Kress, Barrie M. Osborne, Baz Luhrmann,..."
1,150000000,9543,62.169881,335154643,116.0,6.2,2317,"{'id': [12, 14, 28, 10749], 'name': ['Adventur...","{'id': [1241, 1965, 12653, 12654, 12655, 41645...",en,Prince of Persia: The Sands of Time,A rogue prince reluctantly joins forces with a...,"{'name': ['Walt Disney Pictures', 'Jerry Bruck...","{'iso_3166_1': ['US'], 'name': ['United States...",2010-05-19,"{'iso_639_1': ['en'], 'name': ['English']}",9543,"{'cast_id': [5, 2, 7, 4, 6, 8, 9, 10, 26, 27, ...","{'credit_id': ['567e74d4c3a36860e9008e46', '52...",1,0,0,1,"[Adventure, Fantasy, Action, Romance]","[persia, sandstorm, brother against brother, a...","[Walt Disney Pictures, Jerry Bruckheimer Films]",[United States of America],[en],"[Jake Gyllenhaal, Gemma Arterton, Ben Kingsley...","[Sarah Bradshaw, Jerry Bruckheimer, Mike Stens..."
2,140000000,5174,22.571780,258022233,91.0,6.1,783,"{'id': [28, 35, 80, 53], 'name': ['Action', 'C...","{'id': [1704], 'name': ['ambassador']}",en,Rush Hour 3,After an attempted assassination on Ambassador...,"{'name': ['New Line Cinema'], 'id': [12]}","{'iso_3166_1': ['US'], 'name': ['United States...",2007-08-08,"{'iso_639_1': ['la', 'en', 'fr', 'ja', 'zh'], ...",5174,"{'cast_id': [2, 3, 4, 5, 6, 7, 8, 9, 26, 27, 2...","{'credit_id': ['52fe43fac3a36847f807b5bd', '52...",1,0,1,0,"[Action, Comedy, Crime, Thriller]",[ambassador],[New Line Cinema],[United States of America],"[la, en, fr, ja, zh]","[Chris Tucker, Jackie Chan, Hiroyuki Sanada, M...","[Billy Weber, Mark Helfrich, Jeff Nathanson, R..."
3,145000000,1735,60.034162,401128639,112.0,5.2,1387,"{'id': [12, 28, 14], 'name': ['Adventure', 'Ac...","{'id': [2280, 3996, 160153, 207372, 13154, 131...",en,The Mummy: Tomb of the Dragon Emperor,"Archaeologist Rick O'Connell travels to China,...","{'name': ['Universal Pictures', 'China Film Co...","{'iso_3166_1': ['DE', 'US'], 'name': ['Germany...",2008-07-01,"{'iso_639_1': ['en', 'zh', 'sa'], 'name': ['En...",1735,"{'cast_id': [1, 2, 8, 12, 13, 14, 15, 16, 17, ...","{'credit_id': ['52fe4312c3a36847f80384c5', '52...",1,0,1,0,"[Adventure, Action, Fantasy]","[emperor, dynasty, dead, quest, world, dominat...","[Universal Pictures, China Film Co-Production ...","[Germany, United States of America]","[en, zh, sa]","[Brendan Fraser, Jet Li, John Hannah, Maria Be...","[Ronna Kress, Stephen Sommers, Sean Daniel, Ro..."
4,27000000,79698,2.418535,0,109.0,4.8,34,"{'id': [28, 12, 878, 10749], 'name': ['Action'...","{'id': [6917, 9840, 3667, 9935, 4414, 196288, ...",en,The Lovers,The Lovers is an epic romance time travel adve...,"{'name': ['Corsan', 'Bliss Media', 'Limelight ...","{'iso_3166_1': ['AU', 'BE', 'IN'], 'name': ['A...",2015-02-13,"{'iso_639_1': ['en'], 'name': ['English']}",79698,"{'cast_id': [11, 13, 22, 17, 14, 15, 16, 18, 1...","{'credit_id': ['52fe49e0c3a368484e145067', '57...",1,0,0,1,"[Action, Adventure, Science 

In [5]:
'''

Function 'keep' creates a dictionary of values which need to be kept. This looks at how often a certain key word
appears/a certain crew member is involved, and keeps the top 500, removing those which are mentioned only a few times

cast (actors) is set to 10, becuase a good actor can only appear in a few movies (e.g. Leonardo DiCaprio), but still
be infulential. Setting the limit to 10 appearances removes only the actors who are likely not influential

'''


def keep(df, columns):
    total = []
    selected_dict = {}
    for column in columns:
        for i in range(len(df)):
            total += df[column][i]
        frequency = list(Counter(total).values())
        frequency.sort(reverse = True)
        try:
            stop = frequency[500]
            if column == 'cast_name':
                stop = 10
        except IndexError:
            stop = min(frequency)
        count = Counter(total)
        selected = [k for k,v in count.items() if v >= stop]
        selected_dict[column] = selected
    return selected_dict

to_keep = keep(data_copy, new_columns)

In [7]:
"""

function below iterates through the dataframe and limits the keywords/crew/cast as specified above

"""



def limit_features(df, columns, to_keep):
    for column in columns: #iterate thorugh each column
        for i in range(len(df)): #iterate through each row
            content = df[column][i] #get a list contained in that cell
            keep = [] #initiate a list of objects to keep
            for element in content: #iterate through the list in the cell
                if element in to_keep[column]: #check if the element is specified as a value we want to keep (top 500)
                    keep.append(element)
            df.at[i, column] = keep
    return df
                        
data_copy = limit_features(data_copy, new_columns, to_keep)
data_copy.head()

,budget,id,popularity,revenue,runtime,vote_average,vote_count,genres,keywords,original_language,original_title,overview,production_companies,production_countries,release_date,spoken_languages,movie_id,cast,crew,lead_male,lead_female,lead2_male,lead2_female,genres_name,keywords_name,production_companies_name,production_countries_name,spoken_languages_name,cast_name,crew_name
0,105000000,64682,61.196071,351040419,143.0,7.3,3769,"{'id': [18, 10749], 'name': ['Drama', 'Romance']}","{'id': [818, 1326, 1523, 3929, 209714], 'name'...",en,The Great Gatsby,An adaptation of F. Scott Fitzgerald's Long Is...,"{'name': ['Village Roadshow Pictures', 'Bazmar...","{'iso_3166_1': ['US', 'AU'], 'name': ['United ...",2013-05-10,"{'iso_639_1': ['en'], 'name': ['English']}",64682,"{'cast_id': [2, 5, 3, 8, 6, 4, 7, 22, 13, 23, ...","{'credit_id': ['52fe46e3c3a368484e0a982d', '52...",1,0,1,0,"[Drama, Romance]","[based on novel, infidelity, obsession, 3d]","[Village Roadshow Pictures, Warner Bros.]","[United States of America, Australia]",[en],"[Leonardo DiCaprio, Tobey Maguire, Carey Mulli...",[Ronna Kress]
1,150000000,9543,62.169881,335154643,116.0,6.2,2317,"{'id': [12, 14, 28, 10749], 'name': ['Adventur...","{'id': [1241, 1965, 12653, 12654, 12655, 41645...",en,Prince of Persia: The Sands of Time,A rogue prince reluctantly joins forces with a...,"{'name': ['Walt Disney Pictures', 'Jerry Bruck...","{'iso_3166_1': ['US'], 'name': ['United States...",2010-05-19,"{'iso_639_1': ['en'], 'name': ['English']}",9543,"{'cast_id': [5, 2, 7, 4, 6, 8, 9, 10, 26, 27, ...","{'credit_id': ['567e74d4c3a36860e9008e46', '52...",1,0,0,1,"[Adventure, Fantasy, Action, Romance]",[based on video game],"[Walt Disney Pictures, Jerry Bruckheimer Films]",[United States of America],[en],"[Jake Gyllenhaal, Ben Kingsley, Alfred Molina,...","[Michael Kahn, Harry Gregson-Williams]"
2,140000000,5174,22.571780,258022233,91.0,6.1,783,"{'id': [28, 35, 80, 53], 'name': ['Action', 'C...","{'id': [1704], 'name': ['ambassador']}",en,Rush Hour 3,After an attempted assassination on Ambassador...,"{'name': ['New Line Cinema'], 'id': [12]}","{'iso_3166_1': ['US'], 'name': ['United States...",2007-08-08,"{'iso_639_1': ['la', 'en', 'fr', 'ja', 'zh'], ...",5174,"{'cast_id': [2, 3, 4, 5, 6, 7, 8, 9, 26, 27, 2...","{'credit_id': ['52fe43fac3a36847f807b5bd', '52...",1,0,1,0,"[Action, Comedy, Crime, Thriller]",[],[New Line Cinema],[United States of America],"[la, en, fr, ja, zh]","[Jackie Chan, Max von Sydow, Philip Baker Hall...",[Roger Birnbaum]
3,145000000,1735,60.034162,401128639,112.0,5.2,1387,"{'id': [12, 28, 14], 'name': ['Adventure', 'Ac...","{'id': [2280, 3996, 160153, 207372, 13154, 131...",en,The Mummy: Tomb of the Dragon Emperor,"Archaeologist Rick O'Connell travels to China,...","{'name': ['Universal Pictures', 'China Film Co...","{'iso_3166_1': ['DE', 'US'], 'name': ['Germany...",2008-07-01,"{'iso_639_1': ['en', 'zh', 'sa'], 'name': ['En...",1735,"{'cast_id': [1, 2, 8, 12, 13, 14, 15, 16, 17, ...","{'credit_id': ['52fe4312c3a36847f80384c5', '52...",1,0,1,0,"[Adventure, Action, Fantasy]","[world, time, help, wife, son, supernatural, p...","[Universal Pictures, Relativity Media]","[Germany, United States of America]","[en, zh]","[Brendan Fraser, Jet Li, Maria Bello, Liam Cun...","[Ronna Kress, Kevin Kaska]"
4,27000000,79698,2.418535,0,109.0,4.8,34,"{'id': [28, 12, 878, 10749], 'name': ['Action'...","{'id': [6917, 9840, 3667, 9935, 4414, 196288, ...",en,The Lovers,The Lovers is an epic romance time travel adve...,"{'name': ['Corsan', 'Bliss Media', 'Limelight ...","{'iso_3166_1': ['AU', 'BE', 'IN'], 'name': ['A...",2015-02-13,"{'iso_639_1': ['en'], 'name': ['English']}",79698,"{'cast_id': [11, 13, 22, 17, 14, 15, 16, 18, 1...","{'credit_id': ['52fe49e0c3a368484e145067', '57...",1,0,0,1,"[Action, Adventure, Science Fiction, Romance]","[romance, time, travel, adventure, film, story...",[],"[Australia, Belgium, India]",[en],[Josh Hartnett],[Brian Cox]


In [9]:
"""

One hot encoding

"""

def encode(df, columns):
    for column in columns: #iterate thorugh each column
        mlb = MultiLabelBinarizer()
        transformed = mlb.fit_transform(df[column])
        classes = list(mlb.classes_)
        for c in range(len(classes)): #add a column for each class and populate it 
            df[classes[c]] = transformed[:, c]


encode(data_copy, new_columns)
data_copy.head()

budget     id  popularity    revenue  runtime  vote_average  vote_count  \
0  105000000  64682   61.196071  351040419    143.0           7.3        3769   
1  150000000   9543   62.169881  335154643    116.0           6.2        2317   
2  140000000   5174   22.571780  258022233     91.0           6.1         783   
3  145000000   1735   60.034162  401128639    112.0           5.2        1387   
4   27000000  79698    2.418535          0    109.0           4.8          34   

                                              genres  \
0  {'id': [18, 10749], 'name': ['Drama', 'Romance']}   
1  {'id': [12, 14, 28, 10749], 'name': ['Adventur...   
2  {'id': [28, 35, 80, 53], 'name': ['Action', 'C...   
3  {'id': [12, 28, 14], 'name': ['Adventure', 'Ac...   
4  {'id': [28, 12, 878, 10749], 'name': ['Action'...   

                                            keywords original_language  \
0  {'id': [818, 1326, 1523, 3929, 209714], 'name'...                en   
1  {'id': [1241, 1965, 12653, 12654, 12655, 41645...                en   
2             {'id': [1704], 'name': ['ambassador']}                en   
3  {'id': [2280, 3996, 160153, 207372, 13154, 131...                en   
4  {'id': [6917, 9840, 3667, 9935, 4414, 196288, ...                en   

                          original_title  \
0                       The Great Gatsby   
1    Prince of Persia: The Sands of Time   
2                            Rush Hour 3   
3  The Mummy: Tomb of the Dragon Emperor   
4                             The Lovers   

                                            overview  \
0  An adaptation of F. Scott Fitzgerald's Long Is...   
1  A rogue prince reluctantly joins forces with a...   
2  After an attempted assassination on Ambassador...   
3  Archaeologist Rick O'Connell travels to China,...   
4  The Lovers is an epic romance time travel adve...   

                                production_companies  \
0  {'name': ['Village Roadshow Pictures', 'Bazmar...   
1  {'name': ['Walt Disney Pictures', 'Jerry Bruck...   
2          {'name': ['New Line Cinema'], 'id': [12]}   
3  {'name': ['Universal Pictures', 'China Film Co...   
4  {'name': ['Corsan', 'Bliss Media', 'Limelight ...   

                                production_countries release_date  \
0  {'iso_3166_1': ['US', 'AU'], 'name': ['United ...   2013-05-10   
1  {'iso_3166_1': ['US'], 'name': ['United States...   2010-05-19   
2  {'iso_3166_1': ['US'], 'name': ['United States...   2007-08-08   
3  {'iso_3166_1': ['DE', 'US'], 'name': ['Germany...   2008-07-01   
4  {'iso_3166_1': ['AU', 'BE', 'IN'], 'name': ['A...   2015-02-13   

                                    spoken_languages  movie_id  \
0         {'iso_639_1': ['en'], 'name': ['English']}     64682   
1         {'iso_639_1': ['en'], 'name': ['English']}      9543   
2  {'iso_639_1': ['la', 'en', 'fr', 'ja', 'zh'], ...      5174   
3  {'iso_639_1': ['en', 'zh', 'sa'], 'name': ['En...      1735   
4         {'iso_639_1': ['en'], 'name': ['English']}     79698   

                                                cast  crew  lead_male  \
0  {'cast_id': [2, 5, 3, 8, 6, 4, 7, 22, 13, 23, ...     0          1   
1  {'cast_id': [5, 2, 7, 4, 6, 8, 9, 10, 26, 27, ...     0          1   
2  {'cast_id': [2, 3, 4, 5, 6, 7, 8, 9, 26, 27, 2...     0          1   
3  {'cast_id': [1, 2, 8, 12, 13, 14, 15, 16, 17, ...     0          1   
4  {'cast_id': [11, 13, 22, 17, 14, 15, 16, 18, 1...     0          1   

   lead_female  lead2_male  lead2_female  \
0            0           1             0   
1            0           0             1   
2            0           1             0   
3            0           1             0   
4            0           0             1   

                                     genres_name  \
0                               [Drama, Romance]   
1          [Adventure, Fantasy, Action, Romance]   
2              [Action, Comedy, Crime, Thriller]   
3                   [Adventure, Action, Fantasy]   
4  [Action, Adventure,

In [10]:
''''

adds all novels to the unified 'book' column

''''



def based_on_book(df):
    for i in range(len(df)):
        if df['based on novel'][i]==1:
            df['book'][i] = 1
        if df['book'][i]==1:
            continue
        if df['based on comic book'][i]==1:
            df['book'][i] = 1
        if df['based on young adult novel'][i]==1:
            df['book'][i] = 1
            
based_on_book(data_copy)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


## Save the final dataframe

In [17]:
data_copy.to_pickle(data_dir+"pre-processed/movie_details_encoded.pkl")